# Capstone Project
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for delivery service company.
This project is targeting Korean student in KAIST(Korea Advanced Institute of Science and Technology). In Korea, many people use food delivering service. Most of restaurants used to hire their own delivery member, but these days, most of restaurants use delivery service applications.

We will assume that we are trying to make 'Drone Delivery Service' and looking for the optimal location for our 'Drone Base'. We are targeting student in KAIST so, our candidates will be the location near KAIST. But we also consider that there are many other restaurants in Daejeon city.


## Data <a name="data"></a>

To solve our Business Problem we need to figure out the factors below
- number and location of restaurants clusters near KAIST
- distance of clusters from KAIST
- number of Chinese restaurants in the clusters (Chinese food is most popular delivery food)

We will use 'Google Maps API geocoding' and 'Foursquare API' to get locations and categories of the restaurants.

In [1]:
google_api_key=''

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = '291 Daehak-ro, Eoeun-dong, Yuseong-gu, Daejeon'
kaistcode = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, kaistcode))

Coordinate of 291 Daehak-ro, Eoeun-dong, Yuseong-gu, Daejeon: [36.3721427, 127.36039]


In [3]:
import shapely.geometry
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('kaistcode longitude={}, latitude={}'.format(kaistcode[1], kaistcode[0]))
x, y = lonlat_to_xy(kaistcode[1], kaistcode[0])
print('kaistcode UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('kaistcode longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
kaistcode longitude=127.36039, latitude=36.3721427
kaistcode UTM X=6630245.909923334, Y=13034867.694239527
kaistcode longitude=127.36038999999957, latitude=36.372142699999365


In [4]:
kaistcode_x, kaistcode_y = lonlat_to_xy(kaistcode[1], kaistcode[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = kaistcode_x - 6000
x_step = 600
y_min = kaistcode_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(kaistcode_x, kaistcode_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [5]:
import folium

In [6]:
map_kaist = folium.Map(location=kaistcode, zoom_start=13)
folium.Marker(kaistcode, popup='KAIST').add_to(map_kaist)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=1, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_kaist) 
    #folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_kaist)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_kaist

In [7]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, kaistcode[0],  kaistcode[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format( kaistcode[0],  kaistcode[1], addr))

Reverse geocoding check
-----------------------
Address of [36.3721427, 127.36039] is: 291 Daehak-ro, Eoeun-dong, Yuseong-gu, Daejeon, South Korea


In [8]:
print('Obtaining location addresses')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    
    if address is None:
        address = 'NULL'
        
    address = address.replace(', South Korea', '') # remove "South Korea"
    addresses.append(address)
print(' done.')

Obtaining location addresses
 done.


In [13]:
addresses[:10]

['산51-16 Sinbong-dong, Yuseong-gu, Daejeon',
 '290-2 Hagi-dong, Yuseong-gu, Daejeon',
 '산42 Hagi-dong, Yuseong-gu, Daejeon',
 '335 Hagi-dong, Yuseong-gu, Daejeon',
 '603 Noeun 2(i)-dong, Yuseong-gu, Daejeon',
 '192-1 Noeun 2(i)-dong, Yuseong-gu, Daejeon',
 '843-7 Noeun 2(i)-dong, Yuseong-gu, Daejeon',
 '산6-6 Jaun-dong, Yuseong-gu, Daejeon',
 '78 Sinbong-dong, Yuseong-gu, Daejeon',
 '산50-3 Sinbong-dong, Yuseong-gu, Daejeon']

In [14]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"산51-16 Sinbong-dong, Yuseong-gu, Daejeon",36.400610,127.332938,6.628446e+06,1.302915e+07,5992.495307
1,"290-2 Hagi-dong, Yuseong-gu, Daejeon",36.397633,127.330402,6.629046e+06,1.302915e+07,5840.376700
2,"산42 Hagi-dong, Yuseong-gu, Daejeon",36.394656,127.327867,6.629646e+06,1.302915e+07,5747.173218
3,"335 Hagi-dong, Yuseong-gu, Daejeon",36.391679,127.325332,6.630246e+06,1.302915e+07,5715.767665
4,"603 Noeun 2(i)-dong, Yuseong-gu, Daejeon",36.388702,127.322798,6.630846e+06,1.302915e+07,5747.173218
5,"192-1 Noeun 2(i)-dong, Yuseong-gu, Daejeon",36.385726,127.320264,6.631446e+06,1.302915e+07,5840.376700
6,"843-7 Noeun 2(i)-dong, Yuseong-gu, Daejeon",36.382749,127.317730,6.632046e+06,1.302915e+07,5992.495307
7,"산6-6 Jaun-dong, Yuseong-gu, Daejeon",36.403300,127.339932,6.627546e+06,1.302967e+07,5855.766389
8,"78 Sinbong-dong, Yuseong-gu, Daejeon",36.400323,127.337395,6.628146e+06,1.302967e+07,5604.462508
9,"산50-3 Sinbong-dong, Yuseong-gu, Daejeon",36.397346,127.334859,6.628746e+06,1.302967e+07,5408.326913


In [15]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare

In [16]:
#Foursquare credentials are defined in hidden cell bellow.
foursquare_client_id =''
foursquare_client_secret =''


In [17]:

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_category = ['4bf58dd8d48988d145941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [23]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    chinese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter=chinese_category)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_chinese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurant[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurant, location_restaurants


restaurants = {}
chinese_restaurant = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurant = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurant, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurant, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [24]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of chinese restaurants:', len(chinese_restaurant))
print('Percentage of chinese restaurants: {:.2f}%'.format(len(chinese_restaurant) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 412
Total number of chinese restaurants: 36
Percentage of chinese restaurants: 8.74%
Average number of restaurants in neighborhood: 1.7005494505494505


In [27]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:5]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4d469768bbb1a1437c2d5e72', '대림가든', 36.395014, 127.326418, '대한민국', 135, False, 6629707.228335192, 13028958.034122545)
('52cbbed3498e65c7dcb73c5b', '대림식당', 36.395027, 127.326426, '유성구 노은로367번길 65-60, 대한민국', 135, False, 6629704.841838274, 13028957.69479696)
('507a46cae4b08aa6bebb6c94', "black'smith", 36.38697356335611, 127.32384355601128, '유성구 송림로 56 (대전노은점), 대전광역시, 대전광역시, 305-358, 대한민국', 187, False, 6631002.732995681, 13029430.340371009)
('4bb17086f964a52090933ce3', '풍전돈까스', 36.38707877785887, 127.31765985488892, '대한민국', 277, False, 6631459.044885883, 13028736.60243752)
('50e8fb9ee4b02662b70a3e05', '장항선 본가 노은점', 36.3831901550293, 127.32054901123047, '대한민국', 167, False, 6631770.998560303, 13029422.227921108)
...
Total: 412


In [28]:
print('List of chinese restaurants')
print('---------------------------')
for r in list(chinese_restaurant.values())[:5]:
    print(r)
print('...')
print('Total:', len(chinese_restaurant))

List of chinese restaurants
---------------------------
('4e9659c2d3e3b374959d55ed', 'Fortune China (포춘차이나)', 36.38008174048167, 127.32662455720842, '유성구 은구비로155번안길 38, 대전광역시, 대전광역시, 대한민국', 216, True, 6631733.518702715, 13030386.84012905)
('4bb2c36deb3e95218481ca0a', '리엔차이', 36.37995266352278, 127.32472124638349, '대한민국', 301, True, 6631896.114522365, 13030188.510328088)
('4c397f721e06d13a9cc5793e', '아이리스전통중화요리', 36.37070925751622, 127.31484746632303, '대한민국', 156, True, 6633912.4724547705, 13029966.821534494)
('4d4cdc6b9ee1a35d6a821fdf', '라차우', 36.367308813402765, 127.31603796002669, '유성구 노은동, 대전 광역시, 대한민국', 216, True, 6634287.008680002, 13030418.813880384)
('502db4c5e4b0bf3e6937a049', '신성짬뽕', 36.391885255103475, 127.3487896266759, '대한민국', 331, True, 6628431.796109995, 13031725.274969108)
...
Total: 36


In [29]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: 
Restaurants around location 103: 시루향기, 천안문, 귀성본가, 두레마을, 굴세상, 굴세상 신성동점, 모리화, 복덩어리 본관점
Restaurants around location 104: 시루향기, 굴세상 신성동점, 두레마을, 굴세상, 모리화
Restaurants around location 105: 
Restaurants around location 106: 일이공 생돈까스
Restaurants around location 107: 일이공 생돈까스
Restaurants around location 108: 도우모, 흥남부두, 숨, 삼촌뻘, 오한순 수제비, 羊画里 양화리, 토시살롱, 안도스시
Restaurants around location 109: 이모네식당, 다래짬뽕, 현대옥 죽동점, 흥남부두
Restaurants around location 110: 


In [30]:
map_kaist = folium.Map(location=kaistcode, zoom_start=13)
folium.Marker(kaistcode, popup='Alexanderplatz').add_to(map_kaist)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_chinese = res[6]
    color = 'red' if is_chinese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_kaist)
map_kaist